In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import spacy
from collections import Counter

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion

In [ ]:
from IPython.display import HTML, display
# only need this for collab to enable output text wrapping
# fix from: https://stackoverflow.com/questions/58890109/line-wrapping-in-collaboratory-google-results
def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

## Bagging to BERT: A tour of applied NLP
### Part 1: Four flavors of bagging
### Table of Contents
* [Data processing](#data)
* [Word Counts](#word)
* [TF-IDF](#tfidf)
* [Topic models](#topic)
* [Word vectors](#vectors)



### Data processing <a class="anchor" id="data"></a>

Up first is some preprocessing.  You'll either need to download the [imdb review data](https://ai.stanford.edu/~amaas/data/sentiment/) and save it to this directory OR download the [processed data](https://drive.google.com/file/d/1oN_fO91IBkDHD_u6WXiUCvhhyNexQDJq/view?usp=sharing).

In [ ]:
# # processing the original data into DataFrame
# # here for reference, don't need to run this if you're using reviews.pkl.gz
# source_path = Path('./aclImdb/')
# #neg_files = source_path.glob('./*/neg/*.txt')
# #pos_files = source_path.glob('./*/pos/*.txt')
# all_files = []
# for f in source_path.glob('./*/*/*.txt'):
#     filename = f.as_posix()
#     if 'unsup' not in filename:
#         # split up into useful components
#         _, split, sent, idx = filename.split('/')
#         idx = int(idx.split('_')[0])
#         all_files.append([idx, split, sent, f.read_text()])
# review_df = pd.DataFrame(all_files)
# review_df.columns = ['idx', 'split', 'label', 'text']
# # some minor html cruft is in here
# review_df['text'] = review_df['text'].str.replace('<br /><br />', '')
# review_df = review_df.to_pickle('reviews.pkl.gz')

In [ ]:
# can skip here if you already have reviews.pkl.gz
review_df = pd.read_pickle('reviews.pkl.gz')
review_df['label'] = review_df['label'] == 'pos'

### Word counts  <a class="anchor" id="data"></a>
A very basic way to use a sanitized list of tokens is to do a word count. This unlocks a lot of insights right off and is an important step in exploratory data analysis in text.

In [ ]:
# take a positive and negative review for examples
neg_review = review_df.loc[~review_df.label].iloc[0]['text']
pos_review = review_df[review_df.label].iloc[0]['text']
print('Negative\n', neg_review, '\n')
print('Positive\n', pos_review)

In [ ]:
# base python word count - split on whitespace, use Counter object)
print(Counter(neg_review.split()))

Already see some things that need to be considered; capitalization treats "The" and "the" differently, words like "the" and "it" dominate counts.

Luckily, scikit-learn's CountVectorizer allows for simple preprocessing like this.

In [ ]:
# scikit-learn's countvectorizer
count = CountVectorizer()
neg_vec = count.fit_transform([neg_review])
neg_vec

`CountVectorizer` outputs a sparse matrix by default.  We can convert that to a normal numpy array and stitch it together with the vocabulary from the `fit()` call.

In [ ]:
print(
    dict(zip(count.get_feature_names_out(), 
             neg_vec.toarray().flatten())))

We can see the defaults have already done some amount of cleaning for us.

#### Deterministic Approach with word counts

Let's try a deterministic approach, using word counts and a list of "positive" vs "negative" words.

In [ ]:
pos_words = ["good", "great", "like", "loved"]
neg_words = ["bad", "awful", "dislike", "hated"]

# we're going to use this train/test split throughout
# we'll also use this seed for consistency
# NOTE: Usually you'll want to do a separate validation set when choosing models/featuresets!
seed = 37
np.random.seed(seed)
pct_train = 0.7
X_train, X_test, y_train, y_test = train_test_split(
    review_df['text'],
    review_df['label'], train_size=pct_train)

cv = CountVectorizer(stop_words='english')
train_vecs = cv.fit_transform(X_train)
feats = cv.get_feature_names_out()
pos_idxs = np.where(np.isin(feats, pos_words))[0]
neg_idxs = np.where(np.isin(feats, neg_words))[0]
train_det_score = train_vecs[:, pos_idxs].sum(1) - train_vecs[:, neg_idxs].sum(1)
# easier for group-level score
train_det_score = pd.Series(np.array(train_det_score).ravel(), 
                            index=X_train.index)

In [ ]:
# our threshold - the average score for negative, that or below = negative
neg_thresh = train_det_score.groupby(review_df['label'].loc[X_train.index]).mean()[False]
test_vecs = cv.transform(X_test).toarray()
test_det_score = test_vecs[:, pos_idxs].sum(1) - test_vecs[:, neg_idxs].sum(1)
det_pred = test_det_score>neg_thresh

In [ ]:
print(
    classification_report(y_pred=det_pred,
                          y_true=y_test))

#### Count Vector + Logistic Regression 
Here we try a count vector with Logistic Regression.  This alleviates the need for chosing an arbitrary set of terms and arbitrary threshold as above.

Here I use scikit-learn's [Pipelines](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html).  I won't try and explain how these work, the docs do a better job.  Suffice to say they're useful for ensuring features and models get built in a consistent way.


In [ ]:
count = CountVectorizer(stop_words='english', min_df=0.01)

count_pipeline = Pipeline(
    steps=[("preprocessor", count),
          ('model', LogisticRegression(max_iter=500, solver='liblinear'))]
)

In [ ]:
np.random.seed(seed)
count_pipeline.fit(X_train, y_train)
count_pipeline.score(X_test, y_test)

In [ ]:
print(
    classification_report(y_pred=count_pipeline.predict(X_test),
                          y_true=y_test))

This is actually really good! 88% of the time we're predicting the right class with this model.  But can we do...better?

### TF-IDF <a class="anchor" id="tfidf"></a>
One thing we notice with count vectors is that all words are being counted the same.  We might want to use a weighting scheme to ensure that words that are more informative about the content are flagged as more important.  One weighting scheme is Term Frequency - Inverse Document Frequency (TF-IDF).

Take as an example some kind of simplistic movie reviews.  We can already tell which words are most relevant to the specific content of each review (i.e. "good", "bad", "great").

In [ ]:
docs = ['The movie was good',
        'The movie was bad',
        'The movie was great']

cv = CountVectorizer()
vecs = cv.fit_transform(docs).toarray()
# we'll use pandas DF for easier display
pd.DataFrame(vecs, columns=cv.get_feature_names_out())

You'll notice that `vecs` contains the term frequencies.  If we use sklearn's `TfidfVectorizer`, it will calculate those term counts and then multiply them by the Inverse Document Frequency (IDF).

In [ ]:
tfidf = TfidfVectorizer()
# we'll use pandas DF for easier display
tfidf_vecs = tfidf.fit_transform(docs).toarray()
tfidf_df = pd.DataFrame(tfidf_vecs, columns=tfidf.get_feature_names_out())
tfidf_df

You can see that the discriminative words have higher weight than the non-discriminative words.  

It's worth noting here - in terms of "separability", having 0 vs 1 (count of "good" vs count of "bad") might actually be better.  But these are highly curated examples - you can imagine cases where good and bad descriptive terms are mixed in a review, you want to capture the words that describe better the "aboutness" of the review.  (Think: "This movie was not bad, it was good!")

Now let's fit our regression as above with TF-IDF vectors.

In [ ]:
# we use binary here to handle longer reviews
tfidf = TfidfVectorizer(stop_words='english', min_df=0.01)

tfidf_pipeline = Pipeline(
    steps=[("preprocessor", tfidf),
          ('model', LogisticRegression(max_iter=500, solver='liblinear'))]
)

In [ ]:
np.random.seed(seed)
tfidf_pipeline.fit(X_train, y_train)
print(f'accuracy: {tfidf_pipeline.score(X_test, y_test)}')
print(
    classification_report(y_pred=tfidf_pipeline.predict(X_test),
                          y_true=y_test))

The performance has not changed here, but depending on how you tune these approaches, you might see some differences.

It's also interesting to see how the different approaches weight different parts of the vocabulary.  In the section below, I look at some of the words whose coefficients in the model have particularly high magnitude (i.e. they're impactful on the decision made by the model).

In [ ]:
# looking at the coefficients on the LR for each model
word_feats = tfidf_pipeline['preprocessor'].get_feature_names_out()
# get the largest by magnitude, stitch together to compare
top = 10
top_tfidf = np.argsort(np.abs(tfidf_pipeline['model'].coef_.flatten()))[-top:]
top_count = np.argsort(np.abs(count_pipeline['model'].coef_.flatten()))[-top:]
coef_df = pd.DataFrame([
    word_feats,
    tfidf_pipeline['model'].coef_.ravel(),
    count_pipeline['model'].coef_.ravel()],
    index=['word', 'tfidf', 'count']).T
# normalize result for compare
coef_df['tfidf'] = coef_df['tfidf'].rank()
coef_df['count'] = coef_df['count'].rank()
coef_df.loc[np.unique(np.concatenate([top_tfidf, top_count]))]

Some interesting words would be "bad" and "boring", which have strong negative associations in the tfidf model, but weaker negative associations in the count model.

We can look at where these two models disagree and draw some conclusions about that.

In [ ]:
# most interesting are where there's the largest disagreement
tfidf_pred = tfidf_pipeline.predict_proba(X_test)[:, 1]
count_pred = count_pipeline.predict_proba(X_test)[:, 1]
# most interested in situations where their prediction is opposite, but tfidf is correct
pos_tfidf = tfidf_pred>=0.5
pos_count = count_pred>=0.5
disagree = pos_tfidf!=pos_count
tfidf_correct = pos_tfidf == y_test
disagree_tfidf_correct = np.argwhere(((disagree)&(tfidf_correct)).ravel()).flatten()
disagree_sort = np.argsort(np.abs(tfidf_pred[disagree_tfidf_correct] - count_pred[disagree_tfidf_correct]))

In [ ]:
# assemble in df
compare_df = pd.DataFrame([tfidf_pred, count_pred, y_test, X_test],
            index=['tfidf_pred', 'count_pred', 'label', 'text']).T
# shorten a bit for display
compare_df['text'] = compare_df['text'].apply(lambda x: x[:500])
# for display - just instances of disagreement where tfidf was correct
compare_df = compare_df.reindex(disagree_tfidf_correct).dropna()
compare_df = compare_df.iloc[disagree_sort]

In [ ]:
# simple way to look at some of these differences
for i in compare_df.tail(n=3).itertuples():
    print(f'tfidf_pred: {i.tfidf_pred>=0.5}')
    print(f'count_pred: {i.count_pred>=0.5}')
    print(f'label: {i.label}')
    print(f'text: {i.text}')

It's difficult to see piecemeal, but it does appear that certain words we associate with negative reviews (e.g. "awful") have a stronger influence on prediction in the TF-IDF model.

### Topic Models <a class="anchor" id="topic"></a>


In [ ]:
def display_components(model, word_features, top_display=5):
    # utility for displaying respresentative words per component for topic models
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        top_words_idx = topic.argsort()[::-1][:top_display]
        top_words = [word_features[i] for i in top_words_idx]
        print(" ".join(top_words))

In [ ]:
# choose the number of components (topics)
n_components = 10
# adding a few tweaks, just based on experimentation
nmf = NMF(n_components=n_components,
          init='nndsvda',
         max_iter=500)
# NMF typically uses tfidf, not word counts
# fit tfidf vectorizer
tfidf = TfidfVectorizer(stop_words='english')
tfidf_vecs = tfidf.fit_transform(review_df['text'])
nmf_vecs = nmf.fit_transform(tfidf_vecs)

Both NMF provides a components matrix which corresponds to the loading of each word on each topic.  Higher values means the word is more relevant to that topic.  With the function below, we can display some of the "representative" words from each topic.

In [ ]:
display_components(nmf, tfidf.get_feature_names_out())

One neat thing here is we can see which reviews load highly, for example, on this "horror" topic (topic 7).

In [ ]:
review_df.iloc[np.argsort(nmf_vecs[:, 7])[-2:]]['text'].values

Will you look at that - we might have a nice way of "categorizing" here.  Let's see how this does with sentiment analysis.

In [ ]:
tfidf = TfidfVectorizer(stop_words='english', min_df=0.01)
nmf = NMF(n_components=n_components,
          init='nndsvda',
          max_iter=500)
clf_nmf_pipeline = Pipeline(
    steps=[("preprocessor", tfidf),
           ("topic", nmf),
          ('model', LogisticRegression(max_iter=500, solver='liblinear'))]
)



In [ ]:
np.random.seed(seed)
clf_nmf_pipeline.fit(X_train, y_train)
print(f'accuracy: {clf_nmf_pipeline.score(X_test, y_test)}')
print(
    classification_report(y_pred=clf_nmf_pipeline.predict(X_test),
                          y_true=y_test))

It's not great, but we have much fewer features.  With some tuning, we might be able to get compareable performance with a much smaller feature vector.

We may also want to combine these topic vectors with a smaller set of word counts.  Again, this requires some tuning.

In [ ]:
tfidf_small = TfidfVectorizer(min_df=0.05, stop_words='english')
nmf_pipeline = Pipeline(
    steps=[("tfdf", tfidf), 
           ("nmf", nmf)]
)
clf_union_pipeline = Pipeline(
    steps=[('feats',
            FeatureUnion(
                [("tfdf", tfidf_small), 
                 ("nmf", nmf_pipeline)])),
          ('model', LogisticRegression(max_iter=500, solver='liblinear'))]
)


In [ ]:
np.random.seed(seed)
clf_union_pipeline.fit(X_train, y_train)
print(f'accuracy: {clf_union_pipeline.score(X_test, y_test)}')
print(
    classification_report(y_pred=clf_union_pipeline.predict(X_test),
                          y_true=y_test))

### Word vectors <a class="anchor" id="vectors"></a>
Our next approach is to include context in the word-level representations.  We'll be bringing SpaCy into the mix here, particularly their "medium" English web model, which uses GloVe embeddings.

In [ ]:
import spacy

In [ ]:
# only need to run this once
!python -m spacy download en_core_web_md

In [ ]:
nlp = spacy.load("en_core_web_md")

In [ ]:

class GloveVectorizer(BaseEstimator, TransformerMixin):
    # this is a custom document transformer for use in the scikit-learn pipeline
    def __init__(self, vectorizer):
        self.vectorizer = vectorizer
        return
    
    def fit(self, X, y=None):
        # extracts GloVe vector for all words in vectorizer vocabulary (WxV)
        self.vectorizer.fit(X)
        vocab = self.vectorizer.vocabulary_
        self.vocab_glove = np.zeros(shape=(len(vocab), 300))
        for token, idx in vocab.items():
            self.vocab_glove[idx] = nlp(token).vector
        return self
    
    def transform(self, X, y=None):
        # create the average GloVe vector for all words in document (D)
        # use vectorizer to create DxW vector
        X_transformed = self.vectorizer.transform(X).toarray()
        # sum of words in D
        # add 1 to avoid divide by zero
        sum_words = (X_transformed.sum(1)+1).reshape(-1, 1)
        # create DxV vectors
        glove_vecs = (X_transformed.dot(self.vocab_glove))/sum_words
        return glove_vecs

In [ ]:
# we use binary here to handle longer reviews
count = CountVectorizer(stop_words='english', min_df=0.01, binary=False)
glove = GloveVectorizer(count)

glove_pipeline = Pipeline(
    steps=[("preprocessor", glove),
          ('model', LogisticRegression(max_iter=500, solver='liblinear'))]
)


In [ ]:
np.random.seed(seed)
glove_pipeline.fit(X_train, y_train)
glove_pipeline.score(X_test, y_test)

In [ ]:
print(
    classification_report(y_pred=glove_pipeline.predict(X_test),
                          y_true=y_test))

In part 2 of the tutorial, we'll be digging into more complex models and breaking out of the "bagging" paradigm.  I've separated that portion, as it will likely require use of a GPU to be effective.